## 파이썬 Generator

### [개발 동기](https://www.python.org/dev/peps/pep-0255/)
    When a producer function has a hard enough job that it requires maintaining state between values produced, most programming languages offer no pleasant and efficient solution beyond adding a callback function to the producer's argument list, to be called with each value produced.
    ...
    There are differences among these, but the basic idea is the same: provide a kind of function that can return an intermediate result ("the next value") to its caller, but maintaining the function's local state so that the function can be resumed again right where it left off. 


Generator는 즉 함수 내부에 **State**를 저장할 수 있는 구조

In [1]:
def simple_generator():
    yield 1
    yield 2
    yield 3

In [2]:
for value in simple_generator():
    print(value)

1
2
3


In [3]:
generator = simple_generator()

In [4]:
next(generator)

1

In [5]:
next(generator)

2

In [6]:
next(generator)

3

In [7]:
next(generator)

StopIteration: 

#### 1) Generator의 기본 예제 , fibonacci

In [8]:
def fib():
    x0 = 0
    x1 = 1
    while True:
        yield x1 
        x0, x1 = x1, x0+x1

In [9]:
generator = fib()

In [10]:
for i in range(10):
    print(next(generator))

1
1
2
3
5
8
13
21
34
55


위 코드를 보면, 이 내부에는 State 정보가 크게 2개, x0,x1가 존재하고, 실행할 때마다 yield가 갱신되는 구조를 가지고 있다.

#### 좀 더 공부하기 좋은 예제, Binary Tree Class

In [11]:
class BinarySearchTree:
    def __init__(self, label, left=None, right=None):
        self.label = label
        self.left = left
        self.right = right
    
    def __repr__(self, level=0, indent='\t'):
        s = level*indent + self.label
        if self.left:
            s +=f"\n {self.left.__repr__(level+1, indent)}"
        if self.right:
            s +=f"\n {self.right.__repr__(level+1, indent)}"
        return s
    
    def __iter__(self):
        if self.left : 
            for node in self.left:
                yield node
        yield self.label
        if self.right : 
            for node in self.right:
                yield node

In [17]:
def build_tree(_list):
    n = len(_list)
    if n == 0:
        return []
    i = n // 2
    return BinarySearchTree(_list[i], build_tree(_list[:i]), build_tree(_list[i+1:]))

In [18]:
tree = build_tree("ABCDEFGHIJK")

In [19]:
for node in tree:
    print(node)

A
B
C
D
E
F
G
H
I
J
K


#### Yield는 Return으로만 쓰이지 않아요, Input으로 받을 수도 있음..!


In [20]:
def double_inputs():
    while True:
        x = yield
        yield x * 2

In [21]:
gen = double_inputs()

In [22]:
next(gen)
gen.send(10) 

20

In [23]:
next(gen)
gen.send(5)

10

##### 좀 더 구체적인 예제를..!

In [24]:
import random

In [25]:
def get_data():
    """Return 3 random integers between 0 and 9"""
    return random.sample(range(10), 3)

def consume():
    """Displays a running average across lists of integers sent to it"""
    running_sum = 0
    data_items_seen = 0

    while True:
        data = yield
        data_items_seen += len(data)
        running_sum += sum(data)
        print('The running average is {}'.format(running_sum / float(data_items_seen)))

def produce(consumer):
    """Produces a set of values and forwards them to the pre-defined consumer
    function"""
    while True:
        data = get_data()
        print('Produced {}'.format(data))
        consumer.send(data)
        yield

In [26]:
consumer = consume()
consumer.send(None)
producer = produce(consumer)

for _ in range(10):
    print('Producing...')
    next(producer)

Producing...
Produced [3, 5, 7]
The running average is 5.0
Producing...
Produced [0, 4, 1]
The running average is 3.3333333333333335
Producing...
Produced [1, 3, 5]
The running average is 3.2222222222222223
Producing...
Produced [6, 0, 7]
The running average is 3.5
Producing...
Produced [7, 4, 9]
The running average is 4.133333333333334
Producing...
Produced [9, 4, 5]
The running average is 4.444444444444445
Producing...
Produced [0, 8, 9]
The running average is 4.619047619047619
Producing...
Produced [3, 7, 1]
The running average is 4.5
Producing...
Produced [2, 6, 4]
The running average is 4.444444444444445
Producing...
Produced [5, 3, 8]
The running average is 4.533333333333333


이것을 올바르게 이해하려면, 필요한 개념들..!

=> 서브루틴, 코루틴, 메인루틴 (비동기 프로그래밍에서 매우 중요한 개념들로, 서버를 다루거나, 병렬처리 할 때, 알면 매우매우 좋은 개념들로 알고 있습니다. 아직 저도 자세히 몰라서, 다음시간에 보다 개념 위주로 정리해 올게요..!)